In [1]:
import rdflib
from rdflib import Graph, URIRef, Literal, BNode
from pandas import DataFrame
from rdflib.plugins.sparql.processor import SPARQLResult, prepareQuery, prepareUpdate
from rdflib.namespace import FOAF, RDF, SDO
from rdflib.namespace import DefinedNamespace, Namespace
from rdflib.term import URIRef

import numpy as np
from random import sample, choice

In [11]:
def sparql_results_to_df(results: SPARQLResult) -> DataFrame:
    """
    Export results from an rdflib SPARQL query into a `pandas.DataFrame`,
    using Python types. See https://github.com/RDFLib/rdflib/issues/1179.
    """
    return DataFrame(
        data=([None if x is None else x.toPython() for x in row] for row in results),
        columns=[str(x) for x in results.vars],
    )

In [2]:
class ErrorRDF(RDF):
    _NS = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-nsERROR#")

class ErrorFOAF(FOAF):
    _NS = Namespace("http://xmlns.com/foaf/0.1/ERROR/")

class ErrorSDO(SDO):
    _NS = Namespace("https://schema.org/ERROR/")

    Human: URIRef 
    HumAn: URIRef 
    homeAddress: URIRef
    

In [17]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("error_foaf", ErrorFOAF)
g.bind("sdo", SDO)
g.bind("error_sdo", ErrorSDO)

bob = URIRef("http://example.org/people/Bob")
linda = URIRef("http://example.org/people/Linda")
error_hans = URIRef("http://exampleorg/Hans")

g.add((bob, RDF.type, FOAF.Person))
g.add((bob, FOAF.name, Literal("Bob")))
g.add((bob, ErrorFOAF.age, Literal(24)))
g.add((bob, FOAF.knows, linda))
g.add((linda, RDF.type, FOAF.Person))
g.add((linda, FOAF.name, Literal("Linda")))
g.add((error_hans, RDF.type, ErrorSDO.HumAn))
g.add((error_hans, ErrorSDO.name, Literal("Hans")))

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://exampleorg/Hans> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> a foaf:Person ;
    foaf:name "Linda" .




In [30]:
q = prepareUpdate(
    """DELETE {
        ?source_id ?p ?o .
    }
    INSERT { 
        ?target_id ?p ?o . 
    }
    WHERE {
        ?source_id ?p ?o .
    }"""
)
g.update(q, initBindings={'source_id': rdflib.URIRef("http://exampleorg/Hans"), 'target_id': rdflib.URIRef("http://exampleorg/Hans1")})
print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://exampleorg/Hans1> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> a foaf:Person ;
    foaf:name "Linda" .




In [22]:
qres = g.query(
        """
        SELECT DISTINCT ?s
        WHERE {
            ?s ?p ?o .
        }
        """
    )

for row in qres:
    print(f"{row.s}")

sparql_results_to_df(qres)

http://exampleorg/Hans
http://example.org/people/Bob
http://example.org/people/Linda


,s
0,http://exampleorg/Hans
1,http://example.org/people/Bob
2,http://example.org/people/Linda


In [ ]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        PREFIX ex: <http://example.org/people/>
        DELETE {
            ex:Linda a ?o .
        }
        INSERT { 
            ex:Linda1 a ?o . 
        }
        WHERE {
            ex:Linda a ?o .
        }
        """
    )

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a foaf:Person .

<http://exampleorg/Hans> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [ ]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        DELETE { 
            ?s foaf:knows ?o . 
        }
        INSERT { 
            ?s error_foaf:knows ?o . 
        }
        WHERE {
            ?s foaf:knows ?o .
        }
        """
    )
print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a foaf:Person ;
    error_foaf:age 24 ;
    error_foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a foaf:Person .

<http://exampleorg/Hans> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [ ]:
g.update(
        """
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>
        PREFIX error_foaf: <http://xmlns.com/foaf/0.1/ERROR/>
        DELETE {
            ?s a foaf:Person .
        }
        INSERT { 
            ?s a error_foaf:Human . 
        }
        WHERE {
            ?s a foaf:Person .
        }
        """
    )

print(g.serialize())

@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .
@prefix error_sdo: <https://schema.org/ERROR/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Bob> a error_foaf:Human ;
    error_foaf:age 24 ;
    error_foaf:knows <http://example.org/people/Linda> ;
    foaf:name "Bob" .

<http://example.org/people/Linda1> a error_foaf:Human .

<http://exampleorg/Hans> a error_sdo:HumAn ;
    error_sdo:name "Hans" .

<http://example.org/people/Linda> foaf:name "Linda" .




In [ ]:
semicolon_pos = [pos for pos, char in enumerate(str(g.serialize())) if char == ';']
dot_pos = [pos for pos, char in enumerate(str(g.serialize())) if char == '.']
str_arr = np.array(list(str(g.serialize())))
str_arr[sample(semicolon_pos, 2)] = 'Ä'
str_arr[sample(dot_pos, 2)] = 'Ö'
str("".join(str_arr))

'@prefix error_foaf: <http://xmlns.com/foaf/0.1/ERROR/> .\n@prefix error_sdo: <https://schema.org/ERROR/> .\n@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> Ö\n\n<http://example.org/people/Bob> a error_foaf:Human ;\n    error_foaf:age 24 Ä\n    error_foaf:knows <http://example.org/people/Linda> Ä\n    foaf:name "Bob" .\n\n<http://example.org/people/Linda1> a error_foaf:Human .\n\n<http://exampleorg/Hans> a error_sdo:HumAn ;\n    error_sdo:name "Hans" .\n\n<http://example.org/people/Linda> foaf:name "Linda" Ö\n\n'

In [ ]:
'''q = prepareUpdate(
    "INSERT DATA { <?person1> foaf:knows <?person2> .}",
    initNs = { "foaf": FOAF }
)

tim = rdflib.URIRef("http://www.w3.org/People/Berners-Lee/card#i")
g.update(q, initBindings={'person1': tim, 'person2': bob})

print(g.serialize())'''

'q = prepareUpdate(\n    "INSERT DATA { <?person1> foaf:knows <?person2> .}",\n    initNs = { "foaf": FOAF }\n)\n\ntim = rdflib.URIRef("http://www.w3.org/People/Berners-Lee/card#i")\ng.update(q, initBindings={\'person1\': tim, \'person2\': bob})\n\nprint(g.serialize())'

In [ ]:
# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
#g.parse("https://makg.org/samples/2018-11-09/Affiliations_s.nt")
g.parse("http://www.w3.org/People/Berners-Lee/card")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("No triple found!")
    else:
       print(subj, pred, obj)

https://www.w3.org/People/Berners-Lee/card#i http://schema.org/owns https://timblbot.inrupt.net/profile/card#me
n325869575d3c4a2f9443876df2c6556db5 http://www.w3.org/2006/vcard/ns#postal-code 02139
https://www.w3.org/People/Berners-Lee/card#i http://xmlns.com/foaf/0.1/img https://www.w3.org/Press/Stock/Berners-Lee/2001-europaeum-eighth.jpg
http://www.w3.org/People/Berners-Lee/card http://xmlns.com/foaf/0.1/maker https://www.w3.org/People/Berners-Lee/card#i
http://dig.csail.mit.edu/2005/ajar/ajaw/data#Tabulator http://usefulinc.com/ns/doap#developer https://www.w3.org/People/Berners-Lee/card#i
https://www.w3.org/People/Berners-Lee/card#i http://xmlns.com/foaf/0.1/account http://www.reddit.com/user/timbl/
https://www.w3.org/People/Berners-Lee/card#i http://www.w3.org/2000/10/swap/pim/contact#assistant https://www.w3.org/People/Berners-Lee/card#amy
n325869575d3c4a2f9443876df2c6556db5 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2006/vcard/ns#Work
https://www.w3.org/Pe

In [ ]:
g.serialize(destination="test.ttl", format="turtle")

g = Graph()
g.parse("test.ttl")
print(f"Graph g has {len(g)} statements.")

Graph g has 86 statements.
